<a href="https://colab.research.google.com/github/ssvadla/Research_01/blob/main/EM_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from google.colab import drive

drive.mount('/content/drive')

train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')
train1.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Sentence,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Others
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [2]:
train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
  print(i)
  train = train.append(i)



      Unnamed: 0                                           Sentence    Target
0            659  Appellant had stated to the officers that she ...   Invalid
1           3456  We shall discuss the facts more fully in conne...    Others
2           2043  â€œPerjury is a false statement, either writte...   Invalid
3           3344  The offense is felony theft by false pretext; ...     Issue
4           3231  Numerous contentions urging the commission of ...     Issue
...          ...                                                ...       ...
3471        1561  Therefore, the deletion of reference to the Cl...  Analysis
3472         327  A paraffin test, which was made on the appella...  Analysis
3473        1462  The sole question presented is the sufficiency...     Facts
3474        1169  The answer was in response to the question ask...     Facts
3475        1470  The Stateâ€™s evidence shows that in the sprin...   Invalid

[3476 rows x 3 columns]
      Unnamed: 0                       

In [3]:
train.sort_values("Sentence", inplace = True)
print(len(train))


37711


In [4]:
 new_train = train.drop_duplicates(subset ="Sentence")


In [5]:
train = new_train

In [6]:
train['Target'].unique()

array(['Invalid', 'Rule/Law/Holding', 'Facts', 'Analysis', 'Others',
       'Conclusion', 'Issue'], dtype=object)

In [7]:
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()


array(['Invalid', 'Rule/Law/Holding', 'Facts', 'Analysis', 'Conclusion',
       'Issue'], dtype=object)

In [8]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

(4416, 7374)


In [10]:
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')

test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [11]:

unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
#unlabel.head()

del unlabel['Complete']
del unlabel['Unnamed: 0']

unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

unlabel_1 = unlabel.loc[:100]


def index_reset(unlabel_2):
  unlabel_2.reset_index(inplace=True)
  del unlabel_2['index']
  #print(unlabel_2.head())
  return unlabel_2

unlabel_1 = index_reset(unlabel_1)
unlabel_1_copy = unlabel_1



x_un1 = tfidf_vect.transform(unlabel_1['text'])






[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
unlabel_1['Target']=-1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
import numpy as np
np.unique(unlabel_1['Target'])

array([-1])

In [14]:
train = train.rename(columns={'Sentence':'text'})

In [15]:
 from sklearn.preprocessing import LabelEncoder
 train['Target']= LabelEncoder().fit_transform(train['Target'])

In [16]:
train_and_unlabel =  pd.concat([train,unlabel_1])
print(len(train))
print(len(unlabel_1))
print(len(train_and_unlabel))

4416
101
4517


In [17]:
train_and_unlabel.head()

,Unnamed: 0,text,Target
1364,239.0,"""(I)n the First Amendment area 'government may...",3
1833,185.0,"""... that nowhere in the statute was it stated...",3
2431,415.0,"""Although a statute may be neither vague, over...",5
2245,416.0,"""For even when pursuing a legitimate interest,...",5
1561,25.0,"""If an indictment has been found or accusation...",2


In [18]:

train_and_unlabel.reset_index(inplace=True)

In [19]:
train_and_unlabel.columns

Index(['index', 'Unnamed: 0', 'text', 'Target'], dtype='object')

In [20]:
del train_and_unlabel['index']
del train_and_unlabel['Unnamed: 0']

In [21]:
train_and_unlabel.head()

,text,Target
0,"""(I)n the First Amendment area 'government may...",3
1,"""... that nowhere in the statute was it stated...",3
2,"""Although a statute may be neither vague, over...",5
3,"""For even when pursuing a legitimate interest,...",5
4,"""If an indictment has been found or accusation...",2


In [22]:
train_and_unlabel.tail()

,text,Target
4512,witness question hostile state mannerism attit...,-1
4513,court volition called witness stand tendered m...,-1
4514,neither side permitted lead,-1
4515,state defendant examined developed testimony f...,-1
4516,thus examined m testimony record,-1


In [23]:
train_and_unlabel_vect = tfidf_vect.transform(train_and_unlabel['text'])

In [24]:
train_and_unlabel_vect_df=pd.DataFrame(train_and_unlabel_vect.toarray())


In [25]:
train_and_unlabel_vect_df.shape

(4517, 7374)

In [26]:
from sklearn.naive_bayes import GaussianNB
gnb_classifier = GaussianNB()

gnb_classifier.fit(train_and_unlabel_vect_df, train_and_unlabel['Target'])

GaussianNB(priors=None, var_smoothing=1e-09)

In [27]:
test_pred = gnb_classifier.predict(t_p.toarray())

In [28]:
from sklearn.metrics import classification_report
test['Target']= LabelEncoder().fit_transform(test['Target'])
classification_report_test = classification_report(test['Target'],test_pred,digits=4)
print(classification_report_test)

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.3452    0.3766    0.3602        77
           1     0.3864    0.6538    0.4857        26
           2     0.7148    0.6854    0.6998       267
           3     0.4600    0.2771    0.3459        83
           4     0.1860    0.2353    0.2078        34
           5     0.3333    0.3235    0.3284        34

    accuracy                         0.5202       521
   macro avg     0.3465    0.3645    0.3468       521
weighted avg     0.5438    0.5202    0.5262       521



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
unlabel_to_classify = pd.DataFrame(x_un1.toarray())
unlabel_predict = gnb_classifier.predict(unlabel_to_classify)
len(unlabel_predict)

101

In [30]:
np.unique(unlabel_predict)

array([-1])

In [31]:
type(unlabel_predict)

numpy.ndarray

In [32]:
count = 0
total = 0 
out_pos = []
pos = []
for i in unlabel_predict:
  #print(i)
  if i == -1:
    pos.append(count)
    total = total + 1
  else:
    out_pos.append(count)

  count = count + 1
  

    



In [33]:
print(count)
print(total)

101
101


In [34]:
out_pos

[]

In [35]:
print(pos)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


In [36]:

#unlabel_1 = unlabel_1.loc[pos,:]
RN = unlabel_1.loc[pos,:]

In [37]:
len(RN)

101

In [38]:
type(RN)

pandas.core.frame.DataFrame

In [39]:
RN.columns

Index(['text', 'Target'], dtype='object')

In [40]:
Q =  unlabel_1.loc[out_pos,:]

In [41]:
len(Q)

0

In [42]:
type(Q)

pandas.core.frame.DataFrame

In [43]:
type(Q)

pandas.core.frame.DataFrame

In [44]:
#comment once increases unlabel data
Q = unlabel_1

In [45]:
unlabel_1.head()

,text,Target
0,motion rehearing filed,-1
1,relator accusation killing husband virgil clark,-1
2,order district judge 39th judicial district re...,-1
3,relator husband married number year,-1
4,evidence indicates family relation always plea...,-1


In [46]:
RN['new_Target'] = unlabel_predict

In [47]:
RN.head()

,text,Target,new_Target
0,motion rehearing filed,-1,-1
1,relator accusation killing husband virgil clark,-1,-1
2,order district judge 39th judicial district re...,-1,-1
3,relator husband married number year,-1,-1
4,evidence indicates family relation always plea...,-1,-1


In [48]:
del RN['new_Target']

In [49]:
del train1
del train2
del train3
del train4
del train5
del train6
del train7
del train8
del train9
del train10
del train_highKappa

In [50]:
#RN = unlabel_1

In [51]:
type(unlabel_1)

pandas.core.frame.DataFrame

In [52]:
type(RN)

pandas.core.frame.DataFrame

In [53]:
RN.columns

Index(['text', 'Target'], dtype='object')

In [54]:
train.columns

Index(['Unnamed: 0', 'text', 'Target'], dtype='object')

In [55]:
del train['Unnamed: 0']

In [56]:
#Step 2:

In [57]:
# p_and_RN = pd.concat([train,RN])

In [58]:
# p_and_RN.reset_index(inplace=True)

In [59]:
# p_and_RN.columns

In [60]:
# p_and_RN.head()

In [61]:
# del p_and_RN['index']

In [62]:
# p_and_RN_vect = tfidf_vect.transform(p_and_RN['text'])

In [63]:
# p_and_RN_vect_df=pd.DataFrame(p_and_RN_vect.toarray())


In [64]:
# type(unlabel_1)

In [65]:
# type(RN)

In [66]:
/

()

In [67]:
# Q_vect = tfidf_vect.transform(Q['text'])

In [68]:
# Q_vect_df=pd.DataFrame(Q_vect.toarray())


In [69]:
# import lightgbm as lgb
# lgb_classifier = lgb.LGBMClassifier()
# lgb_classifier.fit(p_and_RN_vect_df, p_and_RN['Target'])

In [70]:
# np.unique(p_and_RN['Target'])

In [71]:
# Q_pred = lgb_classifier.predict(Q_vect_df)

In [72]:
# np.unique(Q_pred)

In [73]:
# count_q = 0
# total_q = 0 
# out_pos_q = []
# pos_q = []
# for i in Q_pred:
#   #print(i)
#   if i == -1:
#     pos_q.append(count_q)
#     total_q = total_q + 1
#   else:
#     out_pos_q.append(count_q)

#   count_q = count_q + 1

In [74]:
# W = Q.loc[pos_q,:]

In [75]:
# if W.empty :
#   pass
# else:
#   Q = Q.loc[total_q,:]
#   RN = pd.concat([RN,W])

In [76]:
#defina Q for loop1

In [77]:
loop_variable = 0

In [78]:
# Q = unlabel_1

In [79]:
#Q.columns

In [80]:
import lightgbm as lgb
for i in range(0,3):
  print(loop_variable)
  p_and_RN = pd.concat([train,RN])
  p_and_RN.reset_index(inplace=True,drop=True)
  p_and_RN_vect = tfidf_vect.transform(p_and_RN['text'])
  p_and_RN_vect_df=pd.DataFrame(p_and_RN_vect.toarray())

  Q_vect = tfidf_vect.transform(Q['text'])
  Q_vect_df=pd.DataFrame(Q_vect.toarray())

  lgb_classifier = lgb.LGBMClassifier()
  lgb_classifier.fit(p_and_RN_vect_df, p_and_RN['Target'])
  np.unique(p_and_RN['Target'])

  Q_pred = lgb_classifier.predict(Q_vect_df)
  np.unique(Q_pred)

  count_q = 0
  total_q = 0 
  out_pos_q = []
  pos_q = []
  for i in Q_pred:
    #print(i)
    if i == -1:
      pos_q.append(count_q)
      total_q = total_q + 1
    else:
      out_pos_q.append(count_q)

    count_q = count_q + 1

  Q.reset_index(inplace=True,drop=True)
  W = Q.loc[pos_q,:]


  if W.empty :
    print("W is empty, came out of loop")
    break
  else:
    Q_new = Q.loc[out_pos_q,:]
    Q = Q_new
    RN = pd.concat([RN,W])
    RN.reset_index(inplace=True,drop=True)
    loop_variable = loop_variable + 1
  print("completed iteration")

0
completed iteration
1
completed iteration
2
W is empty, came out of loop


In [87]:
test_pred =  lgb_classifier.predict(t_p.toarray())

In [88]:
from sklearn.metrics import classification_report
test['Target']= LabelEncoder().fit_transform(test['Target'])
classification_report_test = classification_report(test['Target'],test_pred,digits=4)
print(classification_report_test)

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4000    0.2597    0.3150        77
           1     0.6800    0.6538    0.6667        26
           2     0.6752    0.8876    0.7670       267
           3     0.5636    0.3735    0.4493        83
           4     0.8333    0.2941    0.4348        34
           5     0.6296    0.5000    0.5574        34

    accuracy                         0.6372       521
   macro avg     0.5403    0.4241    0.4557       521
weighted avg     0.6243    0.6372    0.6092       521



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
